In [1]:
% setup MRI-education-resources path and requirements
cd ../
startup

loading image
loading signal


# Accelerated Imaging Methods (In Progress Page)

MRI scans can be accelerated by applying methods that reduce the number of k-space samples required, also known as undersampling.  These require a corresponding advanced image reconstruction method to allow for the reduced sampling.  The techniques covered here include parallel imaging, compressed sensing, and deep learning methods. 

## Learning Goals

1. Describe how images are formed
    * Describe how accelerated imaging methods work
1. Understand the constraints and tradeoffs in MRI
    * Identify accelerated imaging methods
    * Understand when accelerated imaging methods can be applied
1. Manipulate MRI sequence parameters to improve performance
    * Define which parameters are modified when accelerated imaging methods are applied
1. Manipulate and analyze MRI data
    * Reconstruct an image from undersampled raw data


## Formulating the Reconstruction Problem

For image reconstruction, it is helpful to reformulate the MRI Signal Equation, which we can pose for MRI as a linear system and using standard mathematical notation describing systems as:

$$ \mathbf{y} = \mathbf{Ex} + \mathbf{n} $$

where $\mathbf{y}$ is the acquired data, $\mathbf{E}$ is the encoding matrix, $\mathbf{x}$ is the spatial distribution of the transverse magnetization (e.g. image), and $\mathbf{n}$ is noise

The encoding matrix, $E$, must include a discrete Fourier Transform matrix, representing our data is the Fourier Transform of the transverse magnetization, evaluated at k-space locations.  It should also include coil sensitivity profiles in order to support parallel imaging formulations.

## General Formulation of MRI Reconstruction

For advanced image reconstruction methods, it is helpful to reformulate the reconstruction problem as a linear system and using standard mathematical notation describing linear systems as:

$$ \mathbf{y} = \mathbf{Ex} + \mathbf{n} $$

In 2D Cartesian

vectorized data and images

$$ y = \left[ 
    \begin{array}{c}
s_1(t_1) \\
s_1(t_2) \\
\vdots \\
s_1(t_{Nx}) \\
s_2(t_1) \\
\vdots \\
s_{Ny}(t_{Nx})\\
\end{array}
\right]$$


$$ x = \left[ 
    \begin{array}{c}
m(x_1,y_1) \\
m(x_2,y_1) \\
\vdots \\
m(x_{Nx},y_1) \\
m(x_{1},y_2) \\
\vdots \\
m(x_{Nx},y_{Ny}) 
\end{array}
\right]$$


where $\mathbf{y}$ is the acquired data, $\mathbf{E}$ is the encoding matrix, $\mathbf{x}$ is the spatial distribution of the transverse magnetization (e.g. image), and $\mathbf{n}$ is noise

The encoding matrix, $E$, must include a discrete Fourier Transform matrix, representing our data is the Fourier Transform of the transverse magnetization, evaluated at k-space locations.  It should also include coil sensitivity profiles in order to support parallel imaging formulations.

## Parallel Imaging

For parallel imaging (PI), we need to consider the coil sensitivity profiles, $\mathbf{C}$, into encoding matrix along with a Fourier Transform encoding matrix, $\mathbf{F}$:

$$\mathbf{E} = \mathbf{F} \mathbf{C}$$

### Image-space Methods
Image-space parallel imaging methods (e.g. SENSE) can be formulated as the following optimization problem

$$\hat{x}_{PI} = \arg \min_\mathbf{x} \frac{1}{2} \| \mathbf{y} - \mathbf{Ex} \|^2_2$$ 

This can be solved directly by a pseudo-inverse, but requires measurements of the coil sensitivity profiles to calculate $\mathbf{E}$:

$$\hat{x}_{PI} = (\mathbf{E}^H\mathbf{E})^{-1} \mathbf{E}^H \mathbf{y} $$

The k-space sampling patterns used for these methods typically use regular undersampling, meaning there is a consistent pattern of acquired and skipped k-space lines.

### K-space Methods

K-space parallel imaging methods (e.g. GRAPPA) utilize a calibration kernel, computed from the data itself and captured in the matrix $\mathbf{G}$.  These can also be generally formulated as the following optimization problem

$$\hat{x}_{PI} = \arg \min_\mathbf{x} \| \mathbf{y} - \mathbf{Ex} \|^2_2 + \lambda \| (\mathbf{G} - \mathbf{I}) \mathbf{x} \|^2_2 $$

Where $\lambda$ is a tuning or regularization parameter and $\mathbf{I}$ is the identity matrix.  This simultaneous enforces data consistency (first term) as well as self-consistency of the calibration (sescond term).

The calibration kernel, $\mathbf{G}$, can either be directly measured from the data as in GRAPPA, or iteratively estimated during the reconstruction.

The k-space sampling patterns used for these methods typically use regular undersampling, meaning there is a consistent pattern of acquired and skipped k-space lines, combined with full sampling in the center of k-space.

### SNR in Parallel Imaging

Due to ill conditioning of the parallel imaging, there is an SNR penalty when using these methods.  This is characterized by the "g-factor", where $g \geq 1$ characterizes the SNR loss that is dependent on the coil loading and geometry, k-space sampling pattern, and parallel imaging reconstruction method.  When appyling an acceleration factor of $R$, the total readout time reduces the SNR by $\sqrt{R}$ as well, leading to the SNR relationship:

$$SNR_{PI} = \frac{SNR_{full}}{g\sqrt{R}}$$

### Artifacts with Parallel Imaging

Parallel Imaging artifacts typically appear as aliasing.  The undersampled data will have aliasing when not using a parallel imaging reconstruction method, and sometimes the aliasing is not completely removed.


## Compressed Sensing Methods

Compressed Sensing (CS) theory says that an image that is compressible in some domain can then be reconstructed from a subset of data samples.  In other words, we can further accelerate our data acquisition.

Compressed Sensing is formulated as the following optimization problem

$$\hat{x}_{CS} = \arg \min_\mathbf{x} \frac{1}{2} \| \mathbf{y} - \mathbf{Ex} \|^2_2 + \lambda_{CS} \| \mathbf{Wx} \|_1 $$

which includes a data consistency term where the data multiplied by the encoding matrix must match the reconstructed image, and a regularization term that enforces that the image is sparse in some other domain through the sparsifying transform, $\mathbf{W}$.  There is a regularization factor, $\lambda_{CS}$, that must be chosen to balance the data consistency and sparsity terms.

Popular sparsifying transforms include total variation (TV), total generalized variation (TGV), and wavelets.

The k-space sampling patterns used for these methods typically use pseudo-random undersampling with a variable density that preferentially increases the number of samples near the center of k-space.

### SNR in Compressed Sensing

It is difficult to define SNR when using compressed sensing reconstruction methods, as they inherently perform some denoising when constraining the reconstruction based on sparsity.  The apparent SNR can also vary significantly depending on the choice of the regularization factor, $\lambda_{CS}$.

### Artifacts with Compressed Sensing

The most common artifacts from compressed sensing come from overfitting to the sparsity penalty.  This result in either an over-smoothed appearance or, in the case of a wavelet sparsifying transform, blocking artifacts.

## Neural Network/Deep Learning Reconstructions

Convolutional neural networks (CNNs), which form the backbone of deep learning (DL), can be used to convert k-space to image data from a subset of data samples as well.  Conceptually, these methods can be trained to learn how to incorporate coil sensitivity information like parallel imaging and typical image sparsity patterns like compressed sensing.  Since they learn from real-world data, they can learn information that is the most relevant to MRI data and thus have been shown to support higher acceleration factors. 

Perhaps the most popular class of DL reconstruction methods for MRI are so-called "un-rolled" networks.  This term confirms 

Compressed Sensing is formulated as the following optimization problem

$$\hat{x}_{CS} = \arg \min_\mathbf{x} \frac{1}{2} \| \mathbf{y} = \mathbf{Ex} \|^2_2 +\tau \| \mathbf{Wx} \|_1 $$

which includes a data consistency term where the data multiplied by the encoding matrix must match the reconstructed image, and a regularization term that enforces that the image is sparse in some other domain through the sparsifying transform, $\mathbf{W}$.

Popular sparsifying transforms include total variation (TV), total generalized variation (TGV), and wavelets.

The k-space sampling patterns used for these methods typically use pseudo-random undersampling with a variable density that preferentially increases the number of samples near the center of k-space


### SNR in Compressed Sensing

### Artifacts with Compressed Sensing
